In [122]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests, os
from dotenv import load_dotenv
import re
import unidecode
import datetime

In [124]:

load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
PLAYERS_DB_ID = os.getenv("PLAYERS_DB_ID")
TEAMS_DB_ID = os.getenv("TEAMS_DB_ID")
MATCHES_DB_ID = os.getenv("MATCHES_DB_ID")
LA_LIGA_PAGE_ID = os.getenv("LA_LIGA_PAGE_ID")
NOTION_ENDPOINT = "https://api.notion.com/v1/"
BARCA_ID = os.getenv("fcbarcelona")
headers = {
            "accept": "application/json",
            "Notion-Version": "2022-06-28",
            "content-type": "application/json",
            "Authorization": "Bearer " + NOTION_API_KEY 
        }

In [130]:
url= "https://api.notion.com/v1/pages/"+BARCA_ID
data = requests.get(url, headers=headers).json()
data["properties"]["League"]["relation"][0]["id"]


'e1037840-58e3-4e5b-b1a8-aa8a692cae88'

In [117]:
html = urlopen("https://www.fcbarcelona.com/en/football/first-team/players/71253/abde-ezzalzouli")
soup = BeautifulSoup(html, "html")
player_pos_tag   = "div.player-hero__info-meta"
player_name_tag  = "div.player-hero__name"
player_img_tag   = "div.player-hero__img"
player_data_tag  = "div.player-strip__data"
player_promo_tag = "div.content-promo__title"
player_bio_tag   = "div.player-bio__description"

len(soup.select(selector="div.player-strip__content > div.player-strip__info"))


4

'https://www.fcbarcelona.com/en/football/first-team/players/71253/abde-ezzalzouli'

In [ ]:

def find_player_honours(soup):
    player_honour_data = []
    for elem in soup.find_all(attrs={"class":"player-honour"}):
        player_honour_title = elem.find(attrs={"class": "player-honour__title"}).get_text()
        player_honour_dates = elem.find(attrs={"class": "player-honour__dates"}).get_text().split()
        player_honour_dates = [dates for dates in player_honour_dates if len(dates)>1]
        player_honour_title_total = len(player_honour_dates)
        player_honour_dates = " | ".join(player_honour_dates)
        player_honour_data_meta = "{} {}🏆 → {}".format(player_honour_title, player_honour_title_total, player_honour_dates)
        player_honour_data.append(player_honour_data_meta)
    return player_honour_data

for link in soup.find_all(href=re.compile(r"https.*first-team")):
    try:
        player_soup = BeautifulSoup(urlopen(link.attrs["href"]),"html")
        player_soup = player_soup.find(attrs={"class":"teams-page"})
        
        player_data = {}
        
        player_name = find_player_info(player_soup,player_name_tag)
        player_name = re.sub("\n | \s+", " ", player_name).split()
        player_num = int(player_name[0])
        player_name = " ".join(player_name[1:])
        player_pos = find_player_info(player_soup, player_pos_tag)
        player_img = find_player_img(player_soup, player_img_tag)
        player_promo = find_player_info(player_soup, player_promo_tag)
        player_bio = find_player_data(player_soup, player_bio_tag)
        player_bio = " ".join(player_bio)
        
        player_data = find_player_data(player_soup, player_data_tag)
        player_national_team = player_age = player_weight = player_height = ""

        for elem in player_data:
            if re.match(r"\D", elem):
                player_national_team = elem.split(", ")[-1]
            else:
                if re.match(r"\d+/", elem) and int(elem.split("/")[-1])<2008:
                    player_age = 2023-int(elem.split("/")[-1])
                elif re.match(".*kg",elem):
                    player_weight = int(re.sub("kg","", elem))
                elif re.match(".*cm",elem):
                    player_height = int(re.sub("cm","", elem))
        player_honours = find_player_honours(player_soup)
        # print(player_name, player_national_team, player_age, player_pos, player_weight, player_height, player_num, player_img, player_honour_data)
        # create_player(player_name, player_national_team, player_age, player_pos, player_weight, player_height, player_num, player_img, player_promo, player_bio, player_honours)
        
    except:
        print("Data retrieval failed.")